In [1]:
# regular imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import math

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.cluster import KMeans
from scipy import stats
import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr, spearmanr, kruskal

import csv
import cc_acquire
import cc_prepare

In [2]:
df = pd.read_csv('merged_df_with_avg_price.csv',index_col=[0])
df.head()

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history.1,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price
0,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00"
1,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00"
2,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.06,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00"
3,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.06,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00"
4,"101,541.00",Judson College,"Private, nonprofit",AL,36756,Marion,5.00,1.00,3.00,3.00,...,0.24,0.10,0.04,9.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","16,619.00"


In [20]:
location = pd.read_csv('US Zip Codes from 2013 Government Data.csv',index_col=[0])
location['zip_code'] = location.index
location.head()

,LAT,LNG,zip_code
ZIP,,,
601,18.18,-66.75,601
602,18.36,-67.18,602
603,18.46,-67.12,603
606,18.16,-66.93,606
610,18.30,-67.13,610


In [21]:
df.shape

(71422, 126)

In [22]:
location[['zip_code']] = location[['zip_code']].astype(str)

In [7]:
location.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33144 entries, 601 to 99929
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   LAT       33144 non-null  float64
 1   LNG       33144 non-null  float64
 2   zip_code  33144 non-null  object 
dtypes: float64(2), object(1)
memory usage: 1.0+ MB


In [23]:
df = pd.merge(df, location, how = 'left', on = 'zip_code')
df.shape

(71422, 128)

In [ ]:
def add_location(df):
    location = pd.read_csv('zip_lat_long.csv',index_col=[0])
    location['zip_code'] = location.index
    location[['zip_code']] = location[['zip_code']].astype(str)
    df = pd.merge(df, location, how = 'left', on = 'zip_code')
    return df

In [24]:
df.LAT.isnull().sum()

50212

In [27]:
location[location.zip_code == '35762']

,LAT,LNG,zip_code
ZIP,,,


In [28]:
df[df.zip_code == '35762']

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_history.1,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price,LAT,LNG
0,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00",NaN,NaN
1,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00",NaN,NaN
479,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Biology and Life Sciences,"48,851.91","50,004.54","53,463.29","152,319.74","14,444.00",NaN,NaN
480,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Biology and Life Sciences,"48,851.91","50,004.54","53,463.29","152,319.74","14,444.00",NaN,NaN
481,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Biology and Life Sciences,"48,851.91","50,004.54","53,463.29","152,319.74","14,444.00",NaN,NaN
482,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Biology and Life Sciences,"48,851.91","50,004.54","53,463.29","152,319.74","14,444.00",NaN,NaN
483,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Biology and Life Sciences,"48,851.91","50,004.54","53,463.29","152,319.74","14,444.00",NaN,NaN
5811,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,Environment and Natural Resources,"53,669.96","53,782.21","55,281.43","162,733.60","14,444.00",NaN,NaN
8106,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,"Public Affairs, Policy, and Social Work","43,168.30","43,126.21","45,354.10","131,648.61","14,444.00",NaN,NaN
8107,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,153.00,"Public Affairs, Policy, and Social Work","43,168.30","43,126.21","45,354.10","131,648.61","14,444.00",NaN,NaN


## merge with state and city

In [5]:
a = pd.read_csv('worldcities.csv')
a.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.68,139.77,Japan,JP,JPN,Tōkyō,primary,"39,105,000.00",1392685764
1,Jakarta,Jakarta,-6.21,106.85,Indonesia,ID,IDN,Jakarta,primary,"35,362,000.00",1360771077
2,Delhi,Delhi,28.67,77.22,India,IN,IND,Delhi,admin,"31,870,000.00",1356872604
3,Manila,Manila,14.60,120.98,Philippines,PH,PHL,Manila,primary,"23,971,000.00",1608618140
4,São Paulo,Sao Paulo,-23.55,-46.63,Brazil,BR,BRA,São Paulo,admin,"22,495,000.00",1076532519


In [6]:
us = a[a.iso2 == 'US']
us.shape

(7824, 11)

In [7]:
us.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
12,New York,New York,40.69,-73.92,United States,US,USA,New York,NaN,"18,713,220.00",1840034016
27,Los Angeles,Los Angeles,34.11,-118.41,United States,US,USA,California,NaN,"12,750,807.00",1840020491
51,Chicago,Chicago,41.84,-87.69,United States,US,USA,Illinois,NaN,"8,604,203.00",1840000494
91,Miami,Miami,25.78,-80.21,United States,US,USA,Florida,NaN,"6,445,545.00",1840015149
108,Dallas,Dallas,32.79,-96.77,United States,US,USA,Texas,NaN,"5,743,938.00",1840019440


In [8]:
us = us[['city','lat','lng','admin_name']]
us = us.rename(columns={"admin_name": "State"})
us.head()

,city,lat,lng,State
12,New York,40.69,-73.92,New York
27,Los Angeles,34.11,-118.41,California
51,Chicago,41.84,-87.69,Illinois
91,Miami,25.78,-80.21,Florida
108,Dallas,32.79,-96.77,Texas


In [10]:
state = pd.read_csv('csvData.csv')
state = state[['State','Code']]
state.head()

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [24]:
us.shape

(7824, 4)

In [11]:
us = pd.merge(us, state ,how = 'inner', on = 'State')
us.shape

(7824, 5)

In [12]:
us = us.rename(columns={"Code": "state_post_code"})
us = us[['state_post_code','city','lat','lng']]
us.head()

,state_post_code,city,lat,lng
0,NY,New York,40.69,-73.92
1,NY,Brooklyn,40.65,-73.95
2,NY,Queens,40.75,-73.80
3,NY,Manhattan,40.78,-73.97
4,NY,Bronx,40.85,-73.87


In [32]:
us.to_csv('location_info_by_state&city')

## restart merge

In [33]:
df = pd.read_csv('merged_df_with_avg_price.csv',index_col=[0])
df.head()

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history.1,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price
0,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00"
1,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00"
2,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.06,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00"
3,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.06,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00"
4,"101,541.00",Judson College,"Private, nonprofit",AL,36756,Marion,5.00,1.00,3.00,3.00,...,0.24,0.10,0.04,9.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","16,619.00"


In [13]:
df['mark'] = 1
df.head()

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_business_management,deg_percent_awarded_history.1,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price,mark
0,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00",1
1,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00",1
2,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00",1
3,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00",1
4,"101,541.00",Judson College,"Private, nonprofit",AL,36756,Marion,5.00,1.00,3.00,3.00,...,0.10,0.04,9.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","16,619.00",1


In [15]:
df.shape

(71422, 126)

In [16]:
df.tail(50)

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_business_management,deg_percent_awarded_history.1,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price,mark
71372,"180,522.00",Montana State University-Northern,Public,MT,59501,Havre,7.00,1.00,3.00,3.00,...,0.09,0.00,61.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","13,024.00",1
71373,"181,002.00",Creighton University,"Private, nonprofit",NE,68178,Omaha,4.00,1.00,3.00,3.00,...,0.25,0.01,41.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","29,994.00",1
71374,"184,056.00",Lincoln Technical Institute-Iselin,"Private, for-profit",NJ,08830,Iselin,2.00,1.00,1.00,1.00,...,0.00,0.00,NaN,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","23,158.00",1
71375,"188,340.00",Vaughn College of Aeronautics and Technology,"Private, nonprofit",NY,11369,Flushing,2.00,1.00,2.00,2.00,...,0.01,0.00,2.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","39,418.00",1
71376,"196,051.00",SUNY Morrisville,Public,NY,13408,Morrisville,2.00,1.00,2.00,2.00,...,0.09,0.00,228.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","17,469.00",1
71377,"196,680.00",Excelsior College,"Private, nonprofit",NY,12203-5159,Albany,2.00,1.00,3.00,3.00,...,0.15,0.01,NaN,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70",0.00,1
71378,"196,680.00",Excelsior College,"Private, nonprofit",NY,12203-5159,Albany,2.00,1.00,3.00,3.00,...,0.15,0.01,NaN,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70",0.00,1
71379,"197,869.00",Appalachian State University,Public,NC,28608,Boone,5.00,1.00,3.00,3.00,...,0.20,0.01,116.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","14,018.00",1
71380,"199,856.00",Wake Technical Community College,Public,NC,27603-5696,Raleigh,5.00,1.00,1.00,1.00,...,0.29,0.00,"2,327.00",Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","7,429.00",1
71381,"207,847.00",Southeastern Oklahoma State University,Public,OK,74701-3347,Durant,6.00,1.00,3.00,3.00,...,0.15,0.02,184.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","8,508.00",1


In [17]:
df1.tail(50)

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price,mark,lat,lng
71627,"180,522.00",Montana State University-Northern,Public,MT,59501,Havre,7.00,1.00,3.00,3.00,...,61.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","13,024.00",1,48.54,-109.68
71628,"181,002.00",Creighton University,"Private, nonprofit",NE,68178,Omaha,4.00,1.00,3.00,3.00,...,41.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","29,994.00",1,41.26,-96.05
71629,"184,056.00",Lincoln Technical Institute-Iselin,"Private, for-profit",NJ,08830,Iselin,2.00,1.00,1.00,1.00,...,NaN,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","23,158.00",1,40.57,-74.32
71630,"188,340.00",Vaughn College of Aeronautics and Technology,"Private, nonprofit",NY,11369,Flushing,2.00,1.00,2.00,2.00,...,2.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","39,418.00",1,NaN,NaN
71631,"196,051.00",SUNY Morrisville,Public,NY,13408,Morrisville,2.00,1.00,2.00,2.00,...,228.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","17,469.00",1,NaN,NaN
71632,"196,680.00",Excelsior College,"Private, nonprofit",NY,12203-5159,Albany,2.00,1.00,3.00,3.00,...,NaN,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70",0.00,1,42.67,-73.80
71633,"196,680.00",Excelsior College,"Private, nonprofit",NY,12203-5159,Albany,2.00,1.00,3.00,3.00,...,NaN,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70",0.00,1,42.67,-73.80
71634,"197,869.00",Appalachian State University,Public,NC,28608,Boone,5.00,1.00,3.00,3.00,...,116.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","14,018.00",1,36.21,-81.67
71635,"199,856.00",Wake Technical Community College,Public,NC,27603-5696,Raleigh,5.00,1.00,1.00,1.00,...,"2,327.00",Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","7,429.00",1,35.83,-78.64
71636,"207,847.00",Southeastern Oklahoma State University,Public,OK,74701-3347,Durant,6.00,1.00,3.00,3.00,...,184.00,Electrical and Mechanic Repairs and Technologies,"55,800.00","53,429.09","68,582.61","177,811.70","8,508.00",1,34.00,-96.39


In [14]:
df1 = pd.merge(df, us, how = 'left', on = ('state_post_code','city'))
df1.mark.value_counts()

1    71677
Name: mark, dtype: int64

In [18]:
df1.lat.isnull().sum()

11335

In [67]:
# check null value
city_null = df[df.city.isnull()]

In [68]:
city_null.institution_control.value_counts()

Foreign                1741
Private, for-profit     777
Private, nonprofit      352
Public                    6
Name: institution_control, dtype: int64

In [72]:
city_null[city_null.institution_control == 'Foreign'].unit_id_institution.value_counts()

Series([], Name: unit_id_institution, dtype: int64)

In [70]:
city_null.avg_net_price.sum()

0.0

In [75]:
df[df.institution_control == 'Foreign'].value_counts()

Series([], dtype: int64)

In [78]:
df[df.unit_id_institution == '0']

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history.1,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price


In [34]:
us.head()

,state_post_code,city,lat,lng
0,NY,New York,40.69,-73.92
1,NY,Brooklyn,40.65,-73.95
2,NY,Queens,40.75,-73.80
3,NY,Manhattan,40.78,-73.97
4,NY,Bronx,40.85,-73.87


In [35]:
df.shape

(71422, 125)

In [62]:
df1 = pd.merge(df, us, how = 'left', on = ('state_post_code','city'))
df1.shape

(71677, 127)

In [63]:
df.city.isnull().sum()

2876

In [64]:
df.state_post_code.isnull().sum()

2876

In [44]:
# check if there is any duplicate for state and city
us_count = us.groupby(['state_post_code','city']).value_counts()

In [45]:
us_count.to_csv('us_count.csv')

In [54]:
us_count = pd.read_csv('us_count.csv')
us_count.head()

,state_post_code,city,lat,lng,count
0,AK,Anchorage,61.15,-149.11,1
1,AK,Badger,64.80,-147.39,1
2,AK,Bethel,60.79,-161.79,1
3,AK,Chena Ridge,64.79,-148.04,1
4,AK,College,64.87,-147.82,1


In [55]:
us_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7824 entries, 0 to 7823
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state_post_code  7824 non-null   object 
 1   city             7824 non-null   object 
 2   lat              7824 non-null   float64
 3   lng              7824 non-null   float64
 4   count            7824 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 305.8+ KB


In [58]:
us_count['count'].value_counts()

1    7824
Name: count, dtype: int64

# check the null value

In [140]:
df = pd.read_csv('merged_df_with_avg_price.csv',index_col=[0])
df.head()

,unit_id_institution,college_name,institution_control,state_post_code,zip_code,city,region_ipeds,title_IV_eligibility,pred_degree,pred_degree_0and4,...,deg_percent_awarded_health,deg_percent_awarded_business_management,deg_percent_awarded_history.1,non_deg_seeking,major_category,2017,2018,2019,Grand Total,avg_net_price
0,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00"
1,"100,654.00",Alabama A & M University,Public,AL,35762,Normal,5.00,1.00,3.00,3.00,...,0.00,0.16,0.00,153.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","14,444.00"
2,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.06,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00"
3,"100,858.00",Auburn University,Public,AL,36849,Auburn,5.00,1.00,3.00,3.00,...,0.06,0.23,0.01,481.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","23,696.00"
4,"101,541.00",Judson College,"Private, nonprofit",AL,36756,Marion,5.00,1.00,3.00,3.00,...,0.24,0.10,0.04,9.00,Agriculture,"57,605.69","55,517.87","61,388.93","174,512.49","16,619.00"


In [80]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})    
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [82]:
missing_value_sep21 = nulls_by_col(df)
missing_value_sep21.head()

,num_rows_missing,percent_rows_missing
avg_net_price_program,71422,100.00
avg_net_price_other,71422,100.00
med_parent_and_pell,69279,97.00
avg_parent_and_pell,67600,94.65
med_monthly_payment_parent_and_debt,63941,89.53


In [84]:
missing_value_sep21.to_csv('missing_value_sep21.csv')

In [ ]:
cols = ['avg_net_price_program','avg_net_price_other']

In [ ]:
# group 1 invesgation = 2876 missing at the same time
'''
    online_only
    pred_degree
    branch_number
    title_IV_eligibility
    region_ipeds
    zip_code
    state_post_code
    city
'''

In [26]:
group1 = df[df.city.isnull()]

In [27]:
group1.online_only.value_counts()

Series([], Name: online_only, dtype: int64)

In [106]:
nulls_by_col(group1).tail(50)

,num_rows_missing,percent_rows_missing
city,2876,100.00
zip_code,2876,100.00
state_post_code,2876,100.00
non_deg_seeking,2876,100.00
online_only,2876,100.00
comp_rt_ft_150over_expected_time,2876,100.00
comp_rt_ft_150over_expected_time_asian,2876,100.00
comp_rt_ft_150over_expected_time_black,2876,100.00
enrollment_share_black,2876,100.00
enrollment_share_asian,2876,100.00


In [100]:
cols = ['institution_control','Grand Total','avg_net_price','college_name','major_name']
group1.institution_control.value_counts()

Foreign                1741
Private, for-profit     777
Private, nonprofit      352
Public                    6
Name: institution_control, dtype: int64

In [101]:
group1['Grand Total'].sum()

535692705.3079

In [102]:
group1['avg_net_price'].sum()

0.0

In [118]:
group1.non_deg_seeking.isnull().sum()

2876

## drop group1 rows and column ('avg_net_price_program','avg_net_price_other')

In [119]:
df.shape

(71422, 125)

In [141]:
cols = ['avg_net_price_program','avg_net_price_other']
df1 = df.drop(columns = cols)
df1.shape

(71422, 123)

In [142]:
df1 = df1.dropna(subset=['city'])
df1.shape

(68546, 123)

In [122]:
71422 - 68546

2876

In [124]:
nulls_by_col(df1).head(60)

,num_rows_missing,percent_rows_missing
med_parent_and_pell,66426,96.91
avg_parent_and_pell,64784,94.51
med_monthly_payment_parent_and_debt,61165,89.23
med_parent_and_loan,61165,89.23
avg_parent_and_loan,59167,86.32
avg_stafford_and_no_pell_recipients,58618,85.52
avg_stafford_and_pell,58535,85.40
med_stafford_and_pell,57159,83.39
med_stafford_and_no_pell_recipients,57159,83.39
avg_stafford_and_debt,49549,72.29


## merge the location data after delete group 1

In [143]:
df1.shape

(68546, 123)

In [127]:
us.head()

,state_post_code,city,lat,lng
0,NY,New York,40.69,-73.92
1,NY,Brooklyn,40.65,-73.95
2,NY,Queens,40.75,-73.80
3,NY,Manhattan,40.78,-73.97
4,NY,Bronx,40.85,-73.87


In [136]:
us['state_city'] = us.state_post_code + ', ' + us.city
us.head()

,state_post_code,city,lat,lng,state_city
0,NY,New York,40.69,-73.92,"NY, New York"
1,NY,Brooklyn,40.65,-73.95,"NY, Brooklyn"
2,NY,Queens,40.75,-73.80,"NY, Queens"
3,NY,Manhattan,40.78,-73.97,"NY, Manhattan"
4,NY,Bronx,40.85,-73.87,"NY, Bronx"


In [138]:
cols = ['state_post_code','city']
us = us.drop(columns = cols)
us.head()

,lat,lng,state_city
0,40.69,-73.92,"NY, New York"
1,40.65,-73.95,"NY, Brooklyn"
2,40.75,-73.80,"NY, Queens"
3,40.78,-73.97,"NY, Manhattan"
4,40.85,-73.87,"NY, Bronx"


In [145]:
df1['state_city'] = df1.state_post_code + ', ' + df1.city

In [146]:
df2 = pd.merge(df1, us, how = 'left', on = ('state_city'))
df2.shape

(68801, 126)

In [147]:
df2.city.isnull().sum()

0

In [148]:
df2.state_post_code.isnull().sum()

0

In [149]:
68801 - 68546

255